# Regenerative Rocket Engine Design

Date: 4 Dec 2024

Group member 1:
- Name: Biener, Paul
- University ID: 3039001292
- Email: bienerp@berkeley.com

Group member 2:
- Name: Baldua, Dheer
- University ID: 3039302060
- Email: dheerb@berkeley.edu

## Summary and Problem Statement

Regenerative cooling is a critical aspect of rocket engine design. This effectively addresses the challenge of managing extremely high temperatures generated during the combustion process involving the fuel and oxidizer. In this project, we analyze a regenerative rocket engine system that utilizes ethanol as both fuel and coolant, with liquid oxygen as the oxidizer initially. Then, we make a substantial improvement to the engine, using both the fuel and the oxidizer as a coolant by circulating them around the combustion chamber of the rocket engine. 

The problem at hand is to design an efficient cooling circuit that can reliably maintain the structural integrity of the combustion chamber and nozzle walls, whilst maximizing engine performance. Without appropriate thermal management, the immense heat from combustion could potentially severely damage, if not melt, the engine components, leading to a catastrophic and incredibly expensive failure.

Regenerative cooling offers an elegant solution by routing the fuel through cooling channels in the chamber walls prior to injection. This process serves two crucial functions:
1. It absorbs heat from the combustion chamber, preventing overheating of the walls.
2. It preheats the fuel, improving combustion efficiency upon injection.


## Background Information

In this regenerative rocket engine design, we utilize ethanol as the fuel and liquid oxygen (LOX) as the oxidizer. This propellant combination offers several advantages, including low toxicity, room temperature storability, and high energy density. The engine system consists of separate storage tanks for the fuel and oxidizer, connected to the combustion chamber via pumps. The ethanol tank is pressurized to 300 kPa, while the injector feed pressure is set at 1.2 MPa. The fuel and oxidizer mass flow rates are both 0.125 kg/s.

In the combustion chamber, the fuel and oxidizer mix and ignite, producing high-temperature, high-pressure gases. These gases then expand through a convergent-divergent nozzle, accelerating to supersonic speeds and generating thrust.
The nozzle design is critical for optimizing thrust. The converging section accelerates the subsonic flow to sonic speed at the throat, while the diverging section further accelerates the supersonic flow. The nozzle's shape and exit area are carefully designed to maximize thrust and efficiency by ensuring optimal gas expansion.

## Key Assumptions
#### Pump
- Isentropic
- Isothermal
#### Fluid
- Inviscid
- No pressure loss due to turbulent flow
#### Heat 
- Isobaric addition
- 5% heat rejection through walls
- We assume the combustion chamber walls will melt at some point if the heat addition due to combustion is greater than heat being rejected through the walls. To ensure that the combustion chamber does not melt, we calculate the necessary changes in enthalpy required to absorb all of the heat rejected through the walls from the combustion.

### Initial Values
- 1.2 MPa is feed pressure for Injectors
- 300 kPa is tank pressure
- Fuel and Oxidizers are saturated liquids when injected (Given they must be liquid, saturated maximizes ability to absorb heat)
- Ethanol and oxygen mass flow rates of 0.125 kg/second

### Engine Design

![title](System.png)

In [3]:
!pip install CoolProp
!pip install numpy
!pip install matplotlib
import numpy as np
import matplotlib.pyplot as plt
import CoolProp
from CoolProp.CoolProp import PropsSI
from CoolProp.Plots import PropertyPlot

  Using cached CoolProp-6.6.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.3 MB)


### Mass Flow Rate Determination
The paper "[Development History and Verification of the Flight Model of a 500N Ethanol/LOX Rocket Engine](https://www.researchgate.net/publication/323295638_Development_History_and_Verification_of_the_Flight_Model_of_a_500_N_EthanolLOX_Rocket_Engine)" outlines the optimal mass flow rates that can be used to achieve the desired thrust while maintaining stoichiometric or near-stoichiometric combustion. By cross-referencing the performance charts and experimental setup, the flow rates detailed below were determined to be suitable for optimal engine operation.

### Total Mass Flow Rate:
$$
\dot{m}_{\text{total}} = \dot{m}_{\text{fuel}} + \dot{m}_{\text{oxidizer}}
$$

# Total Mass Flow Rate
The total mass flow rate $\dot{m}_{\text{Tot}}$ is the sum of the mass flow rates of ethanol and oxygen:

$$
\dot{m}_{\text{Tot}} = \dot{m}_{\text{E}} + \dot{m}_{\text{O}_2}
$$

Substituting the given values:

$$
\dot{m}_{\text{Tot}} = 0.125 \, \text{kg/s} + 0.125 \, \text{kg/s} = 0.25 \, \text{kg/s}
$$

# Molar Flow Rate
The molar flow rate for a substance can be calculated using its mass flow rate and molar mass:

$$
\dot{n} = \frac{\dot{m}}{M_m}
$$

### For ethanol:
$$
\dot{n}_E = \frac{\dot{m}_E}{M_m(E)} = \frac{0.125}{46.068} \, \text{mol/s}
$$

### For liquid oxygen:
$$
\dot{n}_{O_2} = \frac{\dot{m}_{O_2}}{M_m(O_2)} = \frac{0.125}{32} \, \text{mol/s}
$$



In [4]:
#molar and mass flow rate calculations
mDot_E = 0.125 #[kg/s]
mDot_O2 = 0.125 #[kg/s]
mDot_Tot = mDot_E+mDot_O2
mm_E = 0.046068 #[kg/mol]
mm_O2 = 0.032 #[kg/mol]

print("Total mass flow rate: ", mDot_Tot, "kg/s")


Total mass flow rate:  0.25 kg/s


### Heat released by combustion:
$$
Q_{\text{combustion}} = 1350 \, \text{kJ/mol}
$$

### Heat released per second (flow rate):
$$
Q_{\text{out}} = \left( \frac{\dot{m}_{\text{O}_2}}{M_{\text{O}_2}} \right) \times \frac{1}{3} \times Q_{\text{combustion}}
$$
As oxygen is the limiting reagent, we base our reaction on the amount of oxygen. As 3 oxygen molecules are required to combust 1 ethanol molecule, we divide its molar flow rate by 3 to calculate the correct heat out.

### Heat transferred to the wall:
$$
Q_{\text{inWall}} = Q_{\text{out}} \times 0.05
$$
This equation assumes that 5\% of the heat released is transferred to the wall.

### Conversion of heat to joules:
$$
Q_{\text{inWall\_J}} = Q_{\text{inWall}} \times 1000
$$
This converts the heat into joules (J) for compatibility with other calculations, like those in CoolProp.

In [5]:
#Heat released and into wall
Q_combustion = 1350 #heat released by combustion [kj/mol]
Q_out = ((mDot_O2/mm_O2)/3)* Q_combustion # heat released per second for our flow rate [kj/s]
#assume 10% goes to wall
Q_inWall = Q_out*0.05 # assume 5% goes to wall [kj/kg]
Q_inWall_J = Q_inWall*1000 #convert to [j] for usage with CoolProp
print("Heat into the wall: {0:1.2f} kJ/s".format(Q_inWall))
print("Heat into the wall: {0:1.2f} J/s".format(Q_inWall_J))

Heat into the wall: 87.89 kJ/s
Heat into the wall: 87890.62 J/s


### State Determination: Ethanol Injectors in Feed Lines
Source: [Development History and Verification of the Flight Model of a 500N Ethanol/LOX Rocket Engine](https://www.researchgate.net/publication/323295638_Development_History_and_Verification_of_the_Flight_Model_of_a_500_N_EthanolLOX_Rocket_Engine)

In [6]:
p3_Pa = 1200000 #Pressure for Ethanol Injectors in feed lines https://www.researchgate.net/publication/323295638_Development_History_and_Verification_of_the_Flight_Model_of_a_500_N_EthanolLOX_Rocket_Engine 
x3 = 0
fluid = 'ethanol'
oxidizer = 'oxygen'
print("Enthalpy: {0:1.2f} J/kg".format(PropsSI('H','P',p3_Pa,'Q',x3,fluid)))
print("Temperature: {0:1.2f} K".format(PropsSI('T','P',300000,'H',0,"ethanol")))

Enthalpy: 272155.83 J/kg
Temperature: 351.52 K


In [7]:
#State 3 (pre-injector) properties
h3_JperKg = PropsSI('Hmass','P',p3_Pa,'Q',x3,fluid)
t3_K = PropsSI('T','P',p3_Pa,'Q',x3,fluid)

#State 2 (pre-heat exchanger)
h2_JperKg = h3_JperKg-((Q_inWall_J)/mDot_E) #state 2 enthalpy: steady state so h2 + Q_inWall = h3
s2_JperKgK = PropsSI('S','P', p3_Pa,'Hmass',h2_JperKg,fluid) # state 2 entropy
t2_K = PropsSI('T','P', p3_Pa,'H',h2_JperKg,fluid) #temp before regenerative section

print("Inlet temperature: {0:1.2f}°C".format(t2_K-273.15))
print("Outlet temperature: {0:1.2f}°C".format(t3_K-273.15))

Inlet temperature: -114.59°C
Outlet temperature: 158.06°C


### Work of Ethanol Pump:
$$
W_{\text{EPump}} = \frac{1}{\dot{m}_{\text{E}}} \left( h_2 - h_1 \right)
$$
This equation calculates the work done by the ethanol pump, where $h_{2}$ is the enthalpy at the pump exit and $h_{1}$ is the enthalpy at the pump inlet.

### Work of Oxidizer Pump:
$$
W_{\text{OPump}} = \frac{1}{\dot{m}_{\text{O}_2}} \left( h_{2, \text{O}_2} - h_{1, \text{O}_2} \right)
$$
This equation calculates the work done by the oxidizer pump, where $h_{2\text{O}_2}$ is the enthalpy at the pump exit and $h_{1\text{O}_2}$ is the enthalpy at the pump inlet.



In [8]:
p1_Pa = 300000 #Typical pressure for a low-pressure fuel/oxidizer tank

t1_K = PropsSI('T','P', p1_Pa,'S',s2_JperKgK,fluid)
h1_JperKg = PropsSI('H','P', p1_Pa,'S',s2_JperKgK,fluid)
print("Enthanol tank temperature: {0:1.2f}°C".format(t1_K-273.15))

work_EPump = (1/mDot_E)*(h2_JperKg-h1_JperKg) #Work of ethanol pump in W
print("Fuel pump work: {0:1.2f}W".format(work_EPump))

h2_O2_JperKg = PropsSI('H','P', p3_Pa,'Q',0,oxidizer) #Oxygen at injector
s2_O2_JperKgK = PropsSI('S','P', p3_Pa,'Q',0,oxidizer)

h1_O2_JperKg = PropsSI('H','P', p1_Pa,'S',s2_O2_JperKgK,oxidizer)
T1_O2_K = PropsSI('T','P', p1_Pa,'S',s2_O2_JperKgK,oxidizer)
print("Oxygen temperature: {0:1.2f}°C".format(T1_O2_K-273.15))

work_OPump = (1/mDot_O2)*(h2_O2_JperKg-h1_O2_JperKg) #Work of oxygen pump in W
print("Oxidizer pump work: {0:1.2f}W".format(work_OPump))


Enthanol tank temperature: -114.70°C
Fuel pump work: 7912.98W
Oxygen temperature: -171.13°C
Oxidizer pump work: 34610.82W


### Improvement:
One improvement we can make is assuming that the ethanol only needs to absorb half of the energy into the wall. We assume that cooling with oxygen absorbs the other half. Improvement was chosen as the previous design struggled with heat absorption, as to absorb 5% of the energy released by the combustion, the initial temperature of the ethanol was extremely cold. If half of the combustion energy is absorbed by regenerative cooling with ethanol and the other half is absorbed by regenerative cooling with oxygen, we can have a much more feasible temperature for our inlet.

### Improved Engine Design

![title](System-Improved.png)

In [9]:
Q_inWall = Q_out*0.5
Q_inWall_J = Q_inWall*1000
h2_JperKg = h3_JperKg-((Q_inWall_J*0.05)/mDot_E) #state 2 enthalpy: steady state so h2 + Q_inWall = h3
s2_JperKgK = PropsSI('S','P', p3_Pa,'Hmass',h2_JperKg,fluid) # state 2 entropy
t2_K = PropsSI('T','P', p3_Pa,'H',h2_JperKg,fluid) #temp before regenerative section

print("Inlet temperature: {0:1.2f}°C".format(t2_K-273.15))
print("Outlet temperature: {0:1.2f}°C".format(t3_K-273.15))

t1_K = PropsSI('T','P', p1_Pa,'S',s2_JperKgK,fluid)
h1_JperKg = PropsSI('H','P', p1_Pa,'S',s2_JperKgK,fluid)
print("Enthanol tank temperature: {0:1.2f}°C".format(t1_K-273.15))

work_EPump = (1/mDot_E)*(h2_JperKg-h1_JperKg) #work of ethanol pump in W
print("Fuel pump work: {0:1.2f}W".format(work_EPump))

Inlet temperature: 49.58°C
Outlet temperature: 158.06°C
Enthanol tank temperature: 49.42°C
Fuel pump work: 9420.14W


## Thrust
To calculate the thrust of the rocket, we make several assumptions:
- The exhaust gasses are entirely CO2
- The exhaust pressure at the nozzle outlet is atmospheric pressure
- The exhaust temperature at the nozzle outlet is 973K, the maximum usable temperature of GrCOP-84, multiplied by a safety tolerance of 0.8
- The enthalpy of the fuel and oxidizer at the injectors becomes the enthalpy of the exhaust via the equation 
$$
h_{gas} = \frac{h_{\text{Ethanol},\text{J/kg}} \cdot \dot{m}_E + h_{\text{Oxygen},\text{J/kg}} \cdot \dot{m}_{\text{O}_2}}{\dot{m}_{\text{Tot}}}
$$


The thrust produced by a rocket engine can be expressed as:

$$
F = \dot{m} v_e + (p_e - p_0) A_e
$$

Where:  
- $F$: Thrust (N)  
- $\dot{m}$: Mass flow rate of the exhaust (kg/s)  
- $v_e$: Exhaust velocity relative to the rocket (m/s)  
- $p_e$: Pressure at the nozzle exit (Pa)  
- $p_0$: Ambient pressure (Pa)  
- $A_e$: Area of the nozzle exit (m²)

Because we assume that $p_e = p_0$, this simply becomes:

$$
F = \dot{m} v_e
$$

Our starting point for the analysis was the conditions pre injector, and because the mass flow rates do not change, the heat released by the combustion does not change. Therefore, for both out first and improved analyis, the conditions in the combustion chamber remain the same. This means one thrust analysis is sufficient as both engines will produce the same thrust. Our improvement was chosen to allow the ethanol to be stored at or above room temperature, which would benefit a theoretical rocket by allowing fuel to be stored in a less complex manner.


In [16]:
h1_gas = (h3_JperKg*mDot_E + h2_O2_JperKg*mDot_O2)/mDot_Tot  # Assume the exhaust gas takes on the enthalpy of fuel/oxidizer

h2_gas = h1_gas + (Q_out*1000*0.95)/mDot_Tot


h3_gas = PropsSI('T','P', 101325,'T',973*0.8,"CO2") #Temperature at the exhaust is the max useable temp for GrCOP-84 times 20% safety margin

V = (2*(h2_gas-h3_gas))**0.5
M = V/343

Thrust_N = mDot_Tot*V #P_exhaust = P_ atmosphere so thrust is just Mass*Velocity

print("Exhaust Velocity: {0:1.2f}m/s".format(V))
print("Thrust: {0:1.2f}N ".format(Thrust_N)) #The thrust is on the same order of magnitude as the paper we cited for mass flow rate data

Exhaust Velocity: 3681.79m/s
Thrust: 920.45N 


## Conclusion
The improvement we made to the regenerative rocket engine design, specifically routing the fuel as well as the oxidizer, instead of just the fuel, around the combustion chamber, enabled the ethanol to be stored at a much more reasonable temperature in the storage tank. In the preliminary design, the ethanol needed to be stored at -114°C to ensure it could appropriately and sufficiently absorb all of the combustion chamber’s heat. By using the oxidizer, ethanol needs to be stored at a temperature of 50°C, which is substantially more reasonable than the previous scenario. This design change not only improves the operational efficiency of the engine but also reduces the complexity and energy requirements for cryogenic storage systems. Higher storage temperatures for the ethanol result in a less demanding cryogenic setup, potentially reducing system mass and operational costs.
